In [1]:
from ENDFtk.tree import Tape
from NDSampler import NDSampler, SamplerSettings, generate_covariance_dict
import numpy as np
import glob
import matplotlib.pyplot as plt

endf_tape = Tape.from_file('/home/sole-pie01/ndlib/tendl17-neutron/n-U238-s20.tendl.txt')
covariance_dict = generate_covariance_dict(endf_tape)
del covariance_dict[31]
del covariance_dict[32][151][0] #No RRR
del covariance_dict[33]
del covariance_dict[34]
del covariance_dict[35]
covariance_dict

{32: {151: [1]}}

In [2]:
samplerSettings = SamplerSettings(sampling='LHS', widths_to_reduced=True) #, debug=True)
# covariance_data_Pu9.hdf5
sampler = NDSampler(endf_tape, covariance_dict=covariance_dict, settings=samplerSettings, hdf5_filename="covariance_data_U238.hdf5")
sampler.sample(num_samples = 3) 

Generating 3 samples using LHS method...
Creating tape for sample 1...
Creating tape for sample 2...
Creating tape for sample 3...


In [4]:
endf_tape = Tape.from_file('/home/sole-pie01/ndlib/tendl17-neutron/n-U238-s20.tendl.txt')
MAT = endf_tape.MAT(endf_tape.material_numbers[0])
mf2 = endf_tape.MAT(endf_tape.material_numbers[0]).MF(2).MT(151).parse()
mf2_resonance_range = mf2.isotopes[0].resonance_ranges[1]


mf2_resonance_range.scattering_radius

In [ ]:
import pandas as pd

transformed_df = pd.read_csv('transformed_samples_URR_BW.csv')
transformed_df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The dataset is already loaded as transformed_df
# First row contains nominal values, second row contains uncertainties, rows 3+ contain samples

# Extract the first row (nominal values) and second row (uncertainties)
nominal_values = transformed_df.iloc[0].drop('# Row')
uncertainties = transformed_df.iloc[1].drop('# Row')
sample_means = transformed_df.loc[2].drop('# Row')
sample_stds = transformed_df.loc[3].drop('# Row')
meanpctdiff = transformed_df.iloc[4].drop('# Row')
stdpctdiff = transformed_df.iloc[5].drop('# Row')

# Calculate statistics on the samples (rows 6 and beyond)
actual_samples = transformed_df.iloc[6:].set_index('# Row')

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'Nominal': nominal_values,
    'Sample Mean': sample_means,
    'Mean % Diff': meanpctdiff,
    'Expected STD': uncertainties,
    'Sample STD': sample_stds,
    'STD % Diff': stdpctdiff
})

# Sort by absolute percentage difference in means to highlight the most "off" parameters
comparison_df['Abs Mean % Diff'] = abs(comparison_df['Mean % Diff'])
comparison_df['Abs STD % Diff'] = abs(comparison_df['STD % Diff'])
comparison_df_sorted = comparison_df #.sort_values('Abs Mean % Diff', ascending=False)

# Print the sorted comparison to highlight the parameters with largest deviations
print("Comparison of statistics (sorted by absolute mean % difference):")
pd.set_option('display.float_format', '{:.3f}'.format, 'display.width', 1000)
print(comparison_df_sorted[['Nominal', 'Sample Mean', 'Mean % Diff', 'Expected STD', 'Sample STD', 'STD % Diff']])

# Calculate the average relative difference for means and standard deviations
mean_rel_diff = abs((nominal_values - sample_means) / nominal_values).mean() * 100
std_rel_diff = abs((uncertainties - sample_stds) / uncertainties).mean() * 100

print(f"\nAverage relative difference in means: {mean_rel_diff:.2f}%")
print(f"Average relative difference in standard deviations: {std_rel_diff:.2f}%")

In [ ]:
# Compute the correlation matrix using the samples
corr_matrix = actual_samples.corr()

# Display the correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Sampled Parameters')
plt.tight_layout()
plt.show()
